In [1]:
!pip install -q google-generativeai==0.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 1.8 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
from pathlib import Path

In [3]:
genai.__version__

'0.3.1'

In [4]:
from google.colab import userdata

genai.configure(api_key = userdata.get('Gemini_Key'))

In [5]:
# Set up the model
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [6]:
model = genai.GenerativeModel(model_name = "gemini-pro-vision",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [7]:
def input_image_setup(file_loc):
    if not (img := Path(file_loc)).exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/jpeg",
            "data": Path(file_loc).read_bytes()
            }
        ]
    return image_parts

In [8]:
import json

# ... (your previous code)

def generate_gemini_response(input_prompt, image_loc, question_prompts, output_file_path):
    image_prompt = input_image_setup(image_loc)

    # Create a dictionary to store questions and responses
    responses = {}

    for question_prompt in question_prompts:
        prompt_parts = [input_prompt, image_prompt[0], question_prompt]
        response = model.generate_content(prompt_parts)

        # Use the original question as the key
        responses[question_prompt] = response.text

    # Convert the dictionary to a JSON-formatted string
    json_output = json.dumps(responses, indent=2)

    # Write the JSON data to a file
    with open(output_file_path, 'w') as output_file:
        output_file.write(json_output)

# ... (your remaining code)

# Example usage with multiple questions:
input_prompt = """
               You are an expert in understanding invoices.
               You will receive input images as invoices &
               you will have to answer questions based on the input image
               """
image_loc = "/content/Screenshot (82).png"
question_prompts = ["vendor name", "Invoice number", "Invoice date", "Invoice amount"]
output_file_path = "/content/multiple_questions_responsesss.json"

generate_gemini_response(input_prompt, image_loc, question_prompts, output_file_path)
print(f"Generated responses saved to {output_file_path}")




Generated responses saved to /content/multiple_questions_responsesss.json
